In [ ]:
!pip install sortedcontainers
!pip install tqdm

In [ ]:
!pip install nltk

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

In [ ]:
# Write a data preparation pipeline to convert each email into a feature vector.
# Your preparation pipeline should transform an email into a (sparse) vector
# indicating the presence or absence of each possible word. For example, if all
# emails only ever contain four words, “Hello,” “how,” “are,” “you,” then the email
# “Hello you Hello Hello you” would be converted into a vector [1, 0, 0, 1]
# (meaning [“Hello” is present, “how” is absent, “are” is absent, “you” is
# present]), or [3, 0, 0, 2] if you prefer to count the number of occurrences of
# each word.

df = pd.read_csv("../data/spam_ham_dataset.csv")

# data transformation like taking out \r\n
df['text'] = df['text'].apply(lambda text: text.replace('\r\n', ' '))

stemmer = PorterStemmer()
# stemmer.stem('sophisticated')
corpus = []
# stop words allow to not waste computation on stupid words
# that don't matter
stopwords_set = set(stopwords.words("english"))
for ii in tqdm(range(len(df))):
    # make lowercase
    text: str = df['text'].iloc[ii].lower()
    # take out all punctuation
    text = text.translate(str.maketrans('', '', string.punctuation)).split()
    # bring down to base word --> sophisicated to sophist to minimize word count
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)
    corpus.append(text)

print(len(corpus))
print(df.info())
# X, y = df['text'], df['label_num']

# print(len(df))
# n_train = 4500

# X_train, X_test, y_train, y_test = X[:n_train], X[n_train:], y[:n_train], y[n_train:]

# import numpy as np
# shuffle_index = np.random.permutation(n_train)
# # X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

In [ ]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus).toarray()
y = df['label_num']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)
y_probas_forest = cross_val_predict(clf, X_train, y_train, cv=3,
 method="predict_proba")

In [ ]:
# You may want to add hyperparameters to your preparation pipeline to control
# whether or not to strip off email headers, convert each email to lowercase,
# remove punctuation, replace all URLs with “URL,” replace all numbers with
# “NUMBER,” or even perform stemming (i.e., trim off word endings; there are
# Python libraries available to do this).
# • Then try out several classifiers and see if you can build a great spam classifier,
#   with both high recall and high precision.
print(y_probas_forest)
clf.score(X_test, y_test)

